Chapter 5. Universal Quantification (Forall)
==================

Universal quantification is another core concept in **Prove-It**.  A `Forall` operation, formatted with the $\forall$ symbol, is used to represent universal quantification.  For example, $\forall_x P(x)$ means that $P(x)$ is true for any instance of $x$.  $P(x)$ holds true universally over instances of $x$.  Like `Implies`, `Forall` is a core concept but is defined outside of the core in the `proveit.logic` package. It is known in the core for use in the *specialization* and *generalization* derivation steps discussed below.  First, let us given an example of a `Forall` object.

In [1]:
from proveit.logic import Forall
from proveit._common_ import x, P, Px, Q, Qx, R, Rx, S
%begin universal_quantification

In [2]:
basicForallExpr = Forall(x, Px, conditions=[Qx, Rx], domain=S)

basicForallExpr: forall_{x in S | Q(x) , R(x)} P(x)

The meaning of this **Expression** is that $P(x)$ is a true statement for all instances of $x$ for which $x \in S$ and $Q(x)$ is true.  Let us examine the internal structure of this expression.

In [3]:
basicForallExpr.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,
1,Literal,,
2,Lambda,parameter: 13body: 3conditions: 4,
3,Operation,operator: 5operand: 13,
4,ExprList,"6, 7, 8",
5,Variable,,
6,Operation,operator: 9operands: 10,
7,Operation,operator: 11operand: 13,
8,Operation,operator: 12operand: 13,
9,Literal,,


`Forall` derives from `OperationOverInstances` (`proveit._core_.expression.operation.operation_over_instances.OperationOverInstances` aliased as `proveit.OperationOverInstances`) which generally defines an operation that acts on a **lambda** map with optional conditions.  The idea is like a "functional" (a function of a function).  It operates over the range of instances for the **lambda** parameters for which the condition is satisfied.  Other examples of **expression** types that derive from `OperationOverInstances` are $\exists$, $\sum$, $\prod$.  

In our example above, we see that the $S$ domain is internally represented via the first condition of the conditional **lambda**.  In the external representation, it is displayed more compactly along with the introduction of $x$ before the vertical line that precedes the other conditions.  This is a matter of presentation style that is independent of how **Prove-It** treats this expression.  As far as **Prove-It** is concerned, $x \in S$ is simply a condition no different than $Q(x)$ and $R(x)$.  The various parts of the `Forall` **expression** may be accessed as follows:

In [4]:
# Variable whose value defines the instance.
basicForallExpr.instanceVar # This attribute only exists when there is only one.

x

In [5]:
basicForallExpr.instanceVars # The list of variables whose values define the instance (may be one or more).

AttributeError: 'Forall' object has no attribute 'instanceVars'

In [ ]:
basicForallExpr.instanceExpr # The expression being quantified over.

In [ ]:
basicForallExpr.conditions # The list of conditions of the universal quantification.

In [ ]:
# Domain of the instance variable.
basicForallExpr.domain # This attribute only exists when there is only one instance variable (otherwise use `domains`).

In [ ]:
# Returns the list of conditions that appear after the vertical line in the notation,
basicForallExpr.explicitConditions()  # exluding the domain condition(s).

Specialization
======

The *specialization* derivation step uses *expression substitution* internally.  The difference is that *specialization* has proof implications and enforces the extra restrictions to justify these proof implications.  It also eliminates one or more of the outer $\forall$ operations.


### Basic Specialization

Let us take our basic/generic example of the `Forall` expression above and specialize it with a particular "instance" expression.  To do so, we will make assumptions to trivially allow this derivation step is taken (just to show how this works).

In [ ]:
from proveit import Function, ExprList
from proveit._common_ import fy
from proveit.logic import InSet
assumptions = ExprList(basicForallExpr, InSet(fy, S), Function(Q, fy), Function(R, fy))

`InSet` is another core concept that is defined outside of the core in `proveit.logic`.  It represents the set membership operation using the $\in$ symbol.  It is needed as a core concept specifically for the purpose of ensuring that universal quantification requirements are met (the "instance" expression must be "in" the domain set).

In [ ]:
basicForallSpec = basicForallExpr.specialize({x:fy}, assumptions=assumptions)
basicForallSpec

We have proven, somewhat trivially, that $P(f(y))$ is true assuming that $\forall_{x \in S~|~Q(x)} P(x)$, $f(y) \in S$, $Q(f(y))$ are all true statements.  Let is take a look at the proof for this statement.

In [ ]:
basicForallSpec.proof()

This indicates that the proof requires a *specialization* step (step 0) and explicitly indicates, in the row under step 0, the mapping being performed (mapping $x$ to $f(y)$).  The subsequent proof steps that are required are simply proofs by assumption.  Specifically, the original `Forall` expression must be true and the conditions must be satisfied for the instance $x \mapsto f(y)$: $f(y) \in S$, $Q(f(x))$, and $R(f(y))$.  If any of these are not known to be true under the provided assumptions, this step will fail.  In this example, they are trivially true because we our assumptions were chosen to be precisely what needed to be true for the *specialization* step to succeed.

If we leave out our first assumption, **Prove-It** is unable to prove the original `Forall` expression even after attempting to perform automation and the *specialization* step will fail.

In [ ]:
from proveit import ProofFailure
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[1:])
    assert False, "Expecting an ProofFailure error; should not make it to this point"
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

The automation checks to see of the $S$ **Expression** has a `foldAsForall` method that would automate a proof for universal quantification over $S$.  Since $S$ is a simple **Variable** object, no such automation exists.  Next we'll see what happens when the "instance" is not in the $S$ domain.

In [ ]:
from proveit import SpecializationFailure
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:1]+assumptions[2:])
    assert False, "Expecting an SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

Finally, we demonstrate the case when one of the "explicit" conditions is not met.

In [ ]:
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:2]+assumptions[3:])
    assert False, "Expecting an SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

In [ ]:
try:
    basicForallExpr.specialize({x:fy}, assumptions=assumptions[:3])
    assert False, "Expecting an SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

Also note that you cannot *specialize* a variable that is not one of the `Forall` instance variables.

In [ ]:
basicForallExpr

In [ ]:
try:
    basicForallExpr.specialize({x:fy, Q:R}, assumptions=assumptions)
    assert False, "Expecting an SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

You can *relabel* **Variable**s that are not `Forall` instance variables.  Later we will show that you can *relabel* and *specialize* simultaneously.  You can also *specialize* multiple levels of `Forall` operations simultaneously which is why the previous error message mentions "nested Forall operations". 

### Universal quantification without a domain

It is not necessary to specify a domain in a `Forall` **Expression**.  For example, the condition(s) may provide sufficient restrictions for the universal quantification.  Also, any number of conditions may be specified (including no conditions).

In [ ]:
noDomainForallExpr = Forall(x, Px, conditions=[Qx])

In [ ]:
assert not hasattr(noDomainForallExpr, 'domain') # it should not have a domain attribute

In [ ]:
noDomainForallExpr.exprInfo()

In [ ]:
noDomainForallExpr.specialize({x:fy}, assumptions=[noDomainForallExpr, Function(Q, fy)])

### Lambda scope restrictions

In `tutorial01_core_expr`, we noted scoping restrictions that apply to **Lambda** expressions in the context of *expression substitution*.  That restriction carries over to *specialization* and is very important.  Consider the following example.

In [ ]:
from proveit.logic import NotEquals, Exists
from proveit._common_ import Pxy, y, fy
forallExistsExpr = Forall(x, Exists(y, NotEquals(x, y)))

Note, while `Forall` ($\forall$) has a special meaning in the **Prove-It** core, `Exists` ($\exists$) and `Equals` ($\neq$) do not (they are defined via **axioms** within the `proveit.logic` package which we will explain in a later chapter).  We are using them here to make our point more clear.  Just note that `Exists` is another kind of `OperationOverInstances` that is operates on a **lambda** function:

In [ ]:
Exists(y, Pxy).exprInfo()

If we try to specialize $x$ as $y$ in `nestedForall`, this will fail:

In [ ]:
from proveit import ScopingViolation
try:
    forallExistsExpr.specialize({x:y}, assumptions={forallExistsExpr})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

This should fail.  We cannot derive $\exists_y y \neq y$ by assuming $\forall_{ x } \left[\exists_y x \neq y \right]$.  The former is a stronger statement.  We chose this example, in fact, because the latter can be argued as typically true but the former is never true using reasonable definitions.  Where this goes wrong is in violating the scope of $\exists_y$.  It is introducing $y$ as a new **variable** within the sub-expression $\exists_y x \neq y$.  This **label** is off limits to $x$ which is quantified outside of this sub-expression.  We can *specialize* $x$ to whatever we want as long as we respect these scoping restrictions.  It is not simply $y$ that is off limits; all **expressions** involving $y$ are off limits:

In [ ]:
from proveit import ScopingViolation
try:
    forallExistsExpr.specialize({x:fy}, assumptions={forallExistsExpr})
    assert False, "Expecting an ScopingViolation error; should not make it to this point"
except ScopingViolation as e:
    print("EXPECTED ERROR:", e)

It should also be noted that within a scope, a **variable** may be reused with a different meaning.  This should generally be avoided as it makes **expressions** unclear, but the functionality should be well-defined in case it ever happens.  If this happens, we treat it as a distinct **variable** from anything outside of the scope (that just happens to have the same name).  It can be confusing and should be avoided, but it is well-defined.  For example,

In [ ]:
from proveit.logic import And
redundantInstanceVarExpr = Forall(x, And(Px, Forall(x, Qx)))

In [ ]:
# specializing the outer x does not and should not change the inner x which is treated as a distinct Variable
redundantInstanceVarExpr.specialize({x:fy}, assumptions={redundantInstanceVarExpr})

### Operand, operator, or operation specialization

*Specializing* different parts of an **operation** works essentially the same way as it does with *expression substition*.

We will use the `substitution` axiom of `proveit.logic.equality` for demonstrations in this section out of convenience.  **Axioms** and the `proveit.logic` package will be discussed in more detail later.  For now, we note that **axioms** (and **theorems**) are taken to be true statements without proof as you can see below.

In [ ]:
from proveit.logic.equality._axioms_ import substitution
substitution

In [ ]:
substitution.proof()

Let's grab the $x=y$ condition for use below.

In [ ]:
x_eq_y = substitution.conditions[0]

Operator and operand *specialization* are straightforward:

In [ ]:
from proveit._common_ import f, g
operatorSubstitution = substitution.specialize({f:g}, assumptions=[x_eq_y])

In [ ]:
operatorSubstitution.proof()

Note that $x$ and $y$ mapped to themselves by default.  When a mapping is not specified, the default is to map the **variable** to itself.

In [ ]:
from proveit._common_ import a, b
from proveit.logic import Equals
a_eq_b = Equals(a, b)
operandSubstitution = substitution.specialize({x:a, y:b}, assumptions=[a_eq_b])

In [ ]:
operandSubstitution.proof()

Operation substitution can be done explicitly via a **lambda** expression just as we saw with *expression substitution*.  The **lambda** expression is not literally substituted in; rather, the function that it represents is applied as the operation.

In [ ]:
from proveit import Lambda
from proveit.number import Add
operationSubstitution = substitution.specialize({f:Lambda(x, Add(x, a))}, assumptions=[x_eq_y])

In [ ]:
operationSubstitution.proof()

An alternative way to specify an operation substitution is to map the operation applied to a **variable** onto an **expression** that uses this same **variable**.  In this example, mapping $f(x)$ to $x + a$.  This will be internally translated to the same **lambda** expression as before: $x \mapsto x + a$.

In [ ]:
from proveit._common_ import fx
operationSubstitution2 = substitution.specialize({fx:Add(x, a)}, assumptions=[x_eq_y])

The proof is exactly the same as before.

In [ ]:
operationSubstitution2.proof()

### Specializing multiple levels simultaneously

When `Forall` operations are nested, the universal quantifications may be specialized separately.  For example:

In [ ]:
from proveit._common_ import z, Pxyz
from proveit.number import Less
nestedForall = Forall(x, Forall(y, Forall(z, Pxyz, conditions=[Less(z, Add(x, y))])))

In [ ]:
nestedForallSpec1 = nestedForall.specialize(assumptions=[nestedForall])

In [ ]:
nestedForallSpec2 = nestedForallSpec1.specialize()

In [ ]:
nestedForallSpec3 = nestedForallSpec2.specialize(assumptions=[nestedForallSpec2.conditions[0]])

In [ ]:
nestedForallSpec3.proof()

But for the sake of convenience and efficiency, **Prove-It** also allows nested `Forall` operations to be specialized simultaneously:

In [ ]:
assumptions = ExprList(nestedForall, nestedForallSpec2.conditions[0])
nestedForallSimultaneousSpec = nestedForall.specialize({z:z}, assumptions=assumptions)

We just need to include an explicit mapping for an inner quantified **variable**.  So we indicated above that we want to map $z$ to $z$ even though this is typically the default in order to force it to specialize all three `Forall` operations simultaneously.  The proof is shorter, doing a single all-in-one *specialization*:

In [ ]:
nestedForallSimultaneousSpec.proof()

In the mapping that is indicated below step 0, enumerated set notation (with curly braces) is used to separate the mappings at different levels.  In this way, there can be no ambiguity.  Simultaneous *specialization* can be done for any number of nested levels.

### Specializing and relabeling simultaneously

It is also possible to *relabeling* and *specialization* (over any number of nested levels) in one step.  For example: 

In [ ]:
nestedForallSpecAndRelab = nestedForall.specialize(specializeMap={y:y}, relabelMap={z:a}, assumptions=[nestedForall])

The `relabelMap` is specified separately from the `specializeMap` to be unambiguous.

In [ ]:
nestedForallSpecAndRelab.proof()

The *relabeling* map is always shown after the last comma in the mapping under the *specialization* step.  When any *specialization* occurs, the "step type" is labeled "specialization".  If there is only *relabeling*, the "step type" will indicate "relabeling". 

You are not allowed to specify that the same **variable** is to be *specialized* and *relabeled*.

In [ ]:
assumptions

In [ ]:
nestedForall

In [ ]:
try:
    nestedForall.specialize({y:z}, {y:a}, assumptions=[nestedForall])
    assert False, "Expecting an SpecializationFailure error; should not make it to this point"
except SpecializationFailure as e:
    print("EXPECTED ERROR:", e)

As noted in the previous tutorial chapter, relabeling has another important limitation.  You cannot relabel something using assumptions that involve any of the relabeling variables.  For example, we cannot relabel $P$ to $R$ in `nestedForall` while assuming `nestedForall`.  

In [ ]:
from proveit import RelabelingFailure
try:
    nestedForall.specialize({y:y}, {P:R}, assumptions=[nestedForall])
    assert False, "Expecting an RelabelingFailure error; should not make it to this point"
except RelabelingFailure as e:
    print("EXPECTED ERROR:", e)

### Universal quantification over multiple variables

Rather than nesting `Forall` operations, you can quantify over multiple instance variables for a more succinct expression.

In [ ]:
multiVarForall = Forall((x, y), Pxy, domain=S)

In [ ]:
assumptions = [multiVarForall, InSet(x, S), InSet(y, S)]
multiVarForallSpec = multiVarForall.specialize(assumptions=assumptions)

In [ ]:
multiVarForallSpec.proof()

If you attempt to use the same **variable** multiple times in the list of instance variables, you will get an error.

In [ ]:
try:
    Forall((x, x), Px)
except ValueError as e:
    print('EXPECTED ERROR:', e)

You can also specify different domains for each of the **variables** as a list (or `ExprList`) by setting `domains` rather than `domain`.  The notation will use indicate a cartesian product set.

In [ ]:
multiDomainForall = Forall((x, y), Pxy, domains=[S, R])

However, internally it simply splits off a condition for each *instance variable*.

In [ ]:
multiDomainForall.exprInfo()

In [ ]:
assumptions = [multiDomainForall, InSet(x, S), InSet(y, R)]
multiDomainForallSpec = multiDomainForall.specialize(assumptions=assumptions)

In [ ]:
multiDomainForallSpec.proof()

Universal quantification over an unspecified number of **variables** via **iterations** will be discussed in the chapter on <a href="tutorial11_advanced_proofs.ipynb">proofs using advanced expressions</a>.

Generalization
========

*Generalization* is the "inverse" of *specialization* just as *hypothetical reasoning* was the "inverse" of *modus ponens*.  We can write these derivation rules in a manner that makes this relationship clear (just as we did for *hypothetical reasoning* and *modus ponens*):

Specialization: 
$\begin{array}{c}
\boldsymbol{\vdash} \forall_{x \in S~|~Q(x)} P(x) \\
\hline
\left \{\clubsuit \in S,~Q(\clubsuit) \right \} \boldsymbol{\vdash} P(\clubsuit)
\end{array}$

Generalization: 
$\begin{array}{c}
\left \{ x \in S,~Q(x) \right \} \boldsymbol{\vdash} P(x) \\
\hline
\boldsymbol{\vdash} \forall_{x \in S~|~Q(x)} P(x)
\end{array}$

$P(x)$ and $Q(x)$ are intended to represent any function of $x$.
There is some asymmetry between *specialization* and *generalization*.  $\clubsuit$ here is meant to represent *any* **expression**, not necessarily a **variable**, as long is it does not violate scoping restrictions (e.g., having a free **variable** that is the same as a **lambda** *parameter* within the $P$ or $Q$ functions).  However, *generalization* only applies to an unbound **variable**.  In Prove-It, an unbound **variable** is regarded as an "arbitrary" variable.  Essentially, it is implicitly universally quantified.  Recall that *modus ponens* converts an explicit antecedent to an implicit assumption and *hypothetical reasoning* does the opposite.  Similarly, *specialization* converts an explicit universal quantification to implicit arbitrary variables and *generalization* does the opposite.  The reason for having the explicit and implicit forms is much the same as it was for the antecedent versus assumption.  The explicit form allows nesting but the implicit form provides direct access to the instance expression.  Furthermore, explicit universal quantification offers the power of being able to *specialize* an instance variable to an arbitrary **expression**.

The above derivation rules are expressed for a single **variable**.  Such rules apply more generally to any number of **variables** (including an unspecified number of **variables** via **iterations** discussed in in the chapter on <a href="tutorial11_advanced_proofs.ipynb">proofs using advanced expressions</a>).

Our following examples will start from one of the derived *specialization* instances above.  Specifically:

In [ ]:
operationSubstitution

First, we try to *generalize* this **known truth** for all instances of $a$, $x$, and $y$ without any conditions or domain restrictions:

In [ ]:
from proveit import GeneralizationFailure
try:
    operationSubstitution.generalize((a, x, y))
    assert False, "Expecting an GeneralizationFailure error; should not make it to this point"
except GeneralizationFailure as e:
    print('EXPECTED ERROR:', e)

This fails because the assumptions of the original **known truth** involve the same **variables** that we are trying to *generalize* over.  That is not allowed because universal quantification introduces a new scope for $x$ and $y$ (as well as $a$) and the $x=y$ assumption would be external to this scope.  If, however, this assumption is introduced as a condition of the new universal quantification, then we no longer need to retain it as assumptions.  That assumptions will be absorbed into the universal quantification conditions.

In [ ]:
operationSubstitution.generalize((a, x, y), conditions=[x_eq_y])

Adding additional restrictions, such as a domain and/or extra conditions, only makes the statement weaker and is therefore allowed:

In [ ]:
operationSubstitution.generalize((a, x, y), conditions=[x_eq_y], domain=S)

In [ ]:
operationSubstitution.generalize((a, x, y), conditions=[x_eq_y, Qx], domain=S)

It is also possible to create multiple levels of nested `Forall` operations in one step with possibly different domains.  Simply provide a list of lists of **Variable**s as the first argument to `generalize` and a corresponding list of `domains`.  The conditions are applied at the outermost level possible (as soon as all of the relevant variables have been introduced) but otherwise retaining the order that the `conditions` were supplied.

In [ ]:
Qa = Function(Q, a)
nestedGenExample = operationSubstitution.generalize([[a], [x, y]], domainLists=[[P], [R, S]], conditions=[x_eq_y, Qx, Qa])

Note that the $Q(a)$ condition was moved to the front even though it was the last supplied condition because it can be applied before the others.  The *generalization* with multiple leves of nested `Forall` operations takes one step in the proof:

In [ ]:
nestedGenExample.proof()

For any level in which multiple instance variables are introduced, a domain can be supplied for each new variable (as above) or one may be specified as the same domain for each of them:

In [ ]:
operationSubstitution.generalize([[a], [x, y]], domainLists=[[P], [R]], conditions=[x_eq_y, Qx, Qa])

When *generalizing* over a single **variable**, the first argument may be just that **variable** rather than a list or tuple:

In [ ]:
operationSubstitution.generalize(a, conditions=[Qa])

We may only *generalize* over **variables** (or **iterations** of **variables**) however:

In [ ]:
try:
    operationSubstitution.generalize(Qx, conditions=[Qa])
    assert False, "Expecting an ValueError error; should not make it to this point"    
except ValueError as e:
    print('EXPECTED ERROR:', e)

In [ ]:
try:
    operationSubstitution.generalize([[a], [Qx]], conditions=[Qa])
    assert False, "Expecting an ValueError error; should not make it to this point"
except ValueError as e:
    print('EXPECTED ERROR:', e)

In [ ]:
%end universal_quantification

*Generalizing* an unspecified number of **variables** via an **iteration** will be discussed in the chapter on <a href="tutorial11_advanced_proofs.ipynb">proofs using advanced expressions</a>.

# Next chapter: <a href="tutorial06_theorem_proving.ipynb">Theorem Proving</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>